In [1]:
%run data_loader.ipynb
%run evaluation.ipynb
import xgboost as xgb

white wine columns Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
fixed acidity           1599 non-null float64
volatile acidity        1599 non-null float64
citric acid             1599 non-null float64
residual sugar          1599 non-null float64
chlorides               1599 non-null float64
free sulfur dioxide     1599 non-null float64
total sulfur dioxide    1599 non-null float64
density                 1599 non-null float64
pH                      1599 non-null float64
sulphates               1599 non-null float64
alcohol                 1599 non-null float64
quality                 1599 non-null int64
dtypes: float64(11), int64(1)
memory usage: 150.0 KB
read wine None
<class 'pandas.core.frame.Data

In [6]:
from sklearn.model_selection import train_test_split
features = wines.iloc[:,:-3]
class_labels = np.array(wines['quality_label'])
label_names = ['low', 'medium', 'high']
feature_names = list(features.columns)
X_train, X_test, y_train, y_test = train_test_split(features, class_labels, test_size=0.3, 
                                                    random_state=42)

In [10]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler().fit(X_train)
SX_train = ss.transform(X_train)
SX_test = ss.transform(X_test)

In [12]:
xgb_model = xgb.XGBClassifier(seed=42, max_depth=10, learning_rate=0.3, n_estimaters=100)
xgb_model.fit(SX_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.3, max_delta_step=0,
       max_depth=10, min_child_weight=1, missing=None, n_estimaters=100,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=42, silent=True, subsample=1)

In [13]:
xgb_predictions = xgb_model.predict(SX_test)
display_model_performance_metrics(true_labels=y_test, predicted_labels=xgb_predictions, 
                                  classes=label_names)

Model Performance metrics:
------------------------------
Accuracy: 0.7887
Precision: 0.7891
Recall: 0.7887
F1 Score: 0.7841

Model Classification report:
------------------------------
labels ['low', 'medium', 'high']
             precision    recall  f1-score   support

        low       0.75      0.71      0.73       718
     medium       0.81      0.86      0.83      1178
       high       0.89      0.30      0.44        54

avg / total       0.79      0.79      0.78      1950


Prediction Confusion Matrix:
------------------------------
               Predicted:            
                      low medium high
Actual: low           511    207    0
        medium        165   1011    2
        high            3     35   16


In [ ]:
## Hyper parameters tuning

In [15]:
from sklearn.model_selection import GridSearchCV

param_grid = {
                'n_estimators': [100, 200, 300], 
                'max_depth': [5, 10, 15],
                'learning_rate': [0.3, 0.5]
              }

clf = GridSearchCV(xgb.XGBClassifier(tree_method='exact', seed=42), param_grid, 
                       cv=5, scoring='accuracy')
clf.fit(SX_train, y_train)
print(clf.best_params_)

{'learning_rate': 0.3, 'max_depth': 10, 'n_estimators': 100}


In [16]:
results = clf.cv_results_
for param, score_mean, score_sd in zip(results['params'], results['mean_test_score'], results['std_test_score']):
    print(param, round(score_mean, 4), round(score_sd, 4))

{'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 100} 0.7779 0.0118
{'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 200} 0.7799 0.0066
{'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 300} 0.7788 0.0091
{'learning_rate': 0.3, 'max_depth': 10, 'n_estimators': 100} 0.7871 0.0048
{'learning_rate': 0.3, 'max_depth': 10, 'n_estimators': 200} 0.7845 0.0051
{'learning_rate': 0.3, 'max_depth': 10, 'n_estimators': 300} 0.7799 0.0026
{'learning_rate': 0.3, 'max_depth': 15, 'n_estimators': 100} 0.7823 0.0107
{'learning_rate': 0.3, 'max_depth': 15, 'n_estimators': 200} 0.7818 0.0105
{'learning_rate': 0.3, 'max_depth': 15, 'n_estimators': 300} 0.7805 0.0112
{'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 100} 0.7781 0.0123
{'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 200} 0.7735 0.0091
{'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 300} 0.7748 0.0078
{'learning_rate': 0.5, 'max_depth': 10, 'n_estimators': 100} 0.7768 0.0088
{'learning_rate': 0.5, 'max_dep

In [18]:
xgb_model = xgb.XGBClassifier(seed=42, max_depth=10, learning_rate=0.3, n_estimators=100)
xgb_model.fit(SX_train, y_train)

xgb_predictions = xgb_model.predict(SX_test)
display_model_performance_metrics(true_labels=y_test, predicted_labels=xgb_predictions, 
                                      classes=label_names)

Model Performance metrics:
------------------------------
Accuracy: 0.7887
Precision: 0.7891
Recall: 0.7887
F1 Score: 0.7841

Model Classification report:
------------------------------
labels ['low', 'medium', 'high']
             precision    recall  f1-score   support

        low       0.75      0.71      0.73       718
     medium       0.81      0.86      0.83      1178
       high       0.89      0.30      0.44        54

avg / total       0.79      0.79      0.78      1950


Prediction Confusion Matrix:
------------------------------
               Predicted:            
                      low medium high
Actual: low           511    207    0
        medium        165   1011    2
        high            3     35   16
